# EFPA of a Mobile Cellular Network
Consider the 49-cell cellular network model with the wrapped-around design depicted in the below figure. Choose your own input parameters for the number of channels per cell, arrival rates, and mean holding times. Use a Markov-chain simulation to approximate the overall blocking and dropping probabilities in this network. For the same 49-cell network, obtain the blocking probability using EFPA. Compare the results of the two approaches for a range of parameter values.

![jupyter](./images/cellular_network.png)

## Erlang B Recursion

In [21]:
import numpy as np

In [22]:
def erlang_b_recursion(k, A):
    E = np.zeros(k + 1)
    E[0] = 1
    for m in range(1, k + 1):
        E[m] = A * E[m - 1] / (m + A * E[m - 1])
    return E[k]

## Model Initialization

In [23]:
# total number of the cells
num_cell = 49
# arrival rate of new calls to the cells
lam = np.ones(num_cell) * 6
# call holding rate (service rate) of the cells
mu = 2
# cell dwelling rate
delta = 0.2
# channel number of the cells
k = 10
# Total call arrival rate to the cells
theta = np.ones(num_cell)
# Probability of a handoff from cell i to cell j given that the call will hand off
prob_i_j = np.ones((num_cell, 6)) * (delta / (delta + mu)) / 6
# Blocking Probability for new and handoff calls at the cells
prob_blocking = np.zeros(num_cell)
accuracy = 0.00001

neighbours = {1:[2,7,8,9,46,47],
              2:[1,3,9,10,47,48],
              3:[2,4,10,11,48,49],
              4:[3,4,5,11,12,43,49],
              5:[4,6,12,13,43,44],
              6:[5,7,13,14,44,45],
              7:[1,6,8,14,45,46],
              8:[1,7,9,14,15,21],
              9:[1,2,8,10,15,16],
              10:[2,3,9,11,16,17],
              11:[3,4,10,12,17,18],
              12:[4,5,11,13,18,19],
              13:[5,6,12,14,19,20],
              14:[6,7,8,13,20,21],
              15:[8,9,16,21,22,23],
              16:[9,10,15,17,23,24],
              17:[10,11,16,18,24,25],
              18:[11,12,17,19,25,26],
              19:[12,13,18,20,26,27],
              20:[13,14,19,21,27,28],
              21:[8,14,15,20,22,28],
              22:[15,21,23,28,29,35],
              23:[15,16,22,24,29,30],
              24:[16,17,23,25,30,31],
              25:[17,18,24,26,31,32],
              26:[18,19,25,27,32,33],
              27:[19,20,26,28,33,34],
              28:[20,21,22,27,34,35],
              29:[22,23,30,35,36,37],
              30:[23,24,29,31,37,38],
              31:[24,25,30,32,38,39],
              32:[25,25,31,33,39,40],
              33:[26,27,32,34,40,41],
              34:[27,28,33,35,41,42],
              35:[22,28,29,34,36,42],
              36:[29,35,37,42,43,49],
              37:[29,30,36,38,43,44],
              38:[30,31,37,39,44,45],
              39:[31,32,38,40,45,46],
              40:[32,33,39,41,46,47],
              41:[33,34,40,42,47,48],
              42:[34,35,41,36,48,49],
              43:[4,5,36,37,44,49],
              44:[5,6,37,38,43,45],
              45:[6,7,38,39,44,46],
              46:[1,7,39,40,45,47],
              47:[1,2,40,41,46,48],
              48:[2,3,41,42,47,49],
              49:[3,4,36,42,43,48]}
neighbours_size = 6

## EFPA Simulation
Following the guideline of Slides P504, the EFPA algorithm can be implemented easily.

The parameters of the EFPA are the same as the Markov Chain simulation. They are: $\mu=2$, $\lambda=6$, $\delta=0.2$, $k=10$, $P\left(i,n\right)=\frac{1}{6}$. The accuracy is 0.00001.

The blocking probability obtained by EFPA is 0.0008119904491993213, which is very similar to the results of the Markov Chain simulation. It also lies in the confidence interval of the Markov Chain simulation from *Markov Chain Simulation of a Mobile Cellular Network.ipynb*, which is (0.0007695753518549943, 0.0008408246481450055).

In [24]:
pre = np.zeros(num_cell)
while np.abs(pre - theta).any() > accuracy:
    for i in range(num_cell):
        prob_blocking[i] = erlang_b_recursion(k, theta[i] / (mu + delta))
        neighbour_of_cell_i = np.asarray(neighbours[i + 1]) - 1
        neighbour_theta = theta[neighbour_of_cell_i]
        neighbour_prob_blocking = prob_blocking[neighbour_of_cell_i]
        prob_j_i = (delta / (delta + mu)) / 6
        pre[i] = theta[i]
        theta[i] = lam[i] + np.sum((1 - neighbour_prob_blocking) * neighbour_theta * prob_j_i)

overall_prob_blocking = np.sum(prob_blocking * theta) / np.sum(theta)
print("Blocking Probability (EFPA):", overall_prob_blocking)

Blocking Probability (EFPA): 0.0008119904491993213
